In [1]:
from compile_individual_mol_structures import run_alignment
from MDAnalysis.analysis import rms
import MDAnalysis as mda 
from Bio.PDB import PDBParser, Superimposer
from Bio.PDB.vectors import calc_angle


ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

In [ ]:
u1 = mda.Universe("/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/5MAJ.pdb")       # reference structure
u2 = mda.Universe("/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/aligned_pred_orig.pdb")   # structure to compare

# Select atoms (must match in number and order)
sel = "protein and name CA"
atoms1 = u1.select_atoms(sel)
atoms2 = u2.select_atoms(sel)

In [ ]:
for atom in atoms1:
    print(f"{atom.index}: {atom.resname}{atom.resid} - {atom.name}")

1: ALA1 - CA
6: PRO2 - CA
13: ARG3 - CA
25: SER4 - CA
26: SER4 - CA
36: VAL5 - CA
43: ASP6 - CA
51: TRP7 - CA
65: ARG8 - CA
76: GLU9 - CA
85: LYS10 - CA
94: GLY11 - CA
98: TYR12 - CA
110: VAL13 - CA
117: THR14 - CA
124: PRO15 - CA
131: VAL16 - CA
138: LYS17 - CA
147: ASN18 - CA
155: GLN19 - CA
164: GLY20 - CA
168: GLN21 - CA
169: GLN21 - CA
183: CYS22 - CA
189: GLY23 - CA
193: SER24 - CA
199: CYS25 - CA
205: TRP26 - CA
219: ALA27 - CA
224: PHE28 - CA
235: SER29 - CA
241: ALA30 - CA
246: THR31 - CA
253: GLY32 - CA
257: ALA33 - CA
263: LEU34 - CA
264: LEU34 - CA
278: GLU35 - CA
287: GLY36 - CA
291: GLN37 - CA
300: MET38 - CA
308: PHE39 - CA
319: ARG40 - CA
330: LYS41 - CA
339: THR42 - CA
346: GLY43 - CA
350: ARG44 - CA
361: LEU45 - CA
369: ILE46 - CA
377: SER47 - CA
383: LEU48 - CA
391: SER49 - CA
397: GLU50 - CA
406: GLN51 - CA
415: ASN52 - CA
423: LEU53 - CA
431: VAL54 - CA
438: ASP55 - CA
446: CYS56 - CA
452: SER57 - CA
458: GLY58 - CA
462: PRO59 - CA
469: GLN60 - CA
478: GLY61 - CA
4

In [ ]:
import numpy as np

In [ ]:
# Load structures
u1 = mda.Universe("/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/5MAJ.pdb")       # reference structure
u2 = mda.Universe("/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/aligned_pred_orig.pdb") 

# Get CA atoms with their residue IDs
ca1 = u1.select_atoms("protein and name CA")
ca2 = u2.select_atoms("protein and name CA")

# Build mapping of (chainID, resid) → atom
def build_res_map(ca_atoms):
    return {(a.segid.strip() or a.chainID, a.resid): a for a in ca_atoms}

map1 = build_res_map(ca1)
map2 = build_res_map(ca2)

# Find common residues
common_keys = sorted(set(map1.keys()) & set(map2.keys()))

# Extract positions from both structures for common residues
pos1 = np.array([map1[key].position for key in common_keys])
pos2 = np.array([map2[key].position for key in common_keys])

# Compute RMSD
from MDAnalysis.analysis.rms import rmsd
val = rmsd(pos2, pos1, center=True, superposition=True)
print(f"RMSD over {len(common_keys)} matched residues: {val:.3f} Å")


RMSD over 219 matched residues: 0.046 Å


In [ ]:
for atom in atoms2:
    print(f"{atom.index}: {atom.resname}{atom.resid} - {atom.name}")

1: ALA1 - CA
6: PRO2 - CA
13: ARG3 - CA
24: SER4 - CA
30: VAL5 - CA
37: ASP6 - CA
45: TRP7 - CA
59: ARG8 - CA
70: GLU9 - CA
79: LYS10 - CA
88: GLY11 - CA
92: TYR12 - CA
104: VAL13 - CA
111: THR14 - CA
118: PRO15 - CA
125: VAL16 - CA
132: LYS17 - CA
141: ASN18 - CA
149: GLN19 - CA
158: GLY20 - CA
162: GLN21 - CA
171: CYS22 - CA
177: GLY23 - CA
181: SER24 - CA
187: CYS25 - CA
193: TRP26 - CA
207: ALA27 - CA
212: PHE28 - CA
223: SER29 - CA
229: ALA30 - CA
234: THR31 - CA
241: GLY32 - CA
245: ALA33 - CA
250: LEU34 - CA
258: GLU35 - CA
267: GLY36 - CA
271: GLN37 - CA
280: MET38 - CA
288: PHE39 - CA
299: ARG40 - CA
310: LYS41 - CA
319: THR42 - CA
326: GLY43 - CA
330: ARG44 - CA
341: LEU45 - CA
349: ILE46 - CA
357: SER47 - CA
363: LEU48 - CA
371: SER49 - CA
377: GLU50 - CA
386: GLN51 - CA
395: ASN52 - CA
403: LEU53 - CA
411: VAL54 - CA
418: ASP55 - CA
426: CYS56 - CA
432: SER57 - CA
438: GLY58 - CA
442: PRO59 - CA
449: GLN60 - CA
458: GLY61 - CA
462: ASN62 - CA
470: GLU63 - CA
479: GLY64 - CA

In [ ]:
from Bio.PDB import PDBParser, Superimposer
import numpy as np

def get_ca_atoms_by_residues(structure):
    """Return a dictionary {(chain_id, resid): atom} for CA atoms"""
    ca_atoms = {}
    for model in structure:
        for chain in model:
            for residue in chain:
                if 'CA' in residue:
                    resid = residue.get_id()[1]  
                    chain_id = chain.id.strip()
                    key = (chain_id, resid)
                    ca_atoms[key] = residue['CA']
    return ca_atoms

def compute_rmsd_biopdb(ref_path, pred_path):
    parser = PDBParser(QUIET=True)
    ref_struct = parser.get_structure("ref", ref_path)
    pred_struct = parser.get_structure("pred", pred_path)

    ref_atoms_map = get_ca_atoms_by_residues(ref_struct)
    pred_atoms_map = get_ca_atoms_by_residues(pred_struct)

    # Get common residues
    common_keys = sorted(set(ref_atoms_map) & set(pred_atoms_map))
    print(f"Found {len(common_keys)} common residues for RMSD.")

    if len(common_keys) == 0:
        print("❌ No overlapping residues found.")
        return

    ref_atoms = [ref_atoms_map[key] for key in common_keys]
    pred_atoms = [pred_atoms_map[key] for key in common_keys]

    # Superimpose and calculate RMSD
    sup = Superimposer()
    sup.set_atoms(ref_atoms, pred_atoms)
    print(f"RMSD: {sup.rms:.3f} Å")

    return sup.rms

# Example usage:
ref_pdb = "/path/to/ref.pdb"
pred_pdb = "/path/to/predicted.pdb"
compute_rmsd_biopdb('/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/5MAJ.pdb', '/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/aligned_pred_orig.pdb')


Found 219 common residues for RMSD.
RMSD: 0.580 Å


np.float64(0.5803223603671056)

# measure covalent bond

In [ ]:
pdb_file = "/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/5MAJ.pdb"
pred_file = "/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/aligned_pred_orig.pdb" # only has aligned prediction 

In [ ]:
# get the atoms involved in the covalent bond 
with open(pdb_file, 'r') as pdb:
        for line in pdb:
            if line.startswith("LINK"):
                prot_atom = line[13:17].strip()
                lig_atom = line[43:47].strip()

                res_idx = line[23:27].strip()
                res_name = line[17:21].strip()
                chain_name = line[21:23].strip()
                
                ccd = line[47:50].strip() 
                break 

In [ ]:
# unique protein atom id
prot_atom, res_name, chain_name, res_idx

('SG', 'CYS', 'A', '25')

In [ ]:
# unique lig atom 
lig_atom

'C8'

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolTransforms

mol = Chem.MolFromPDBFile(pred_file, removeHs=False)
conf = mol.GetConformer()

# # Example: Distance between atom 0 and atom 5
# dist = rdMolTransforms.GetBondLength(conf, 0, 5)
# print(f"Distance = {dist:.2f} Å")

In [ ]:
conf

In [ ]:
from Bio.PDB import PDBParser
from rdkit import Chem
from rdkit.Chem import rdmolops

# 1. Parse the PDB file
parser = PDBParser(QUIET=True)
structure = parser.get_structure("complex", pred_file)


In [ ]:
# 2. Get atom lists for protein and ligand separately
protein_atoms = []
ligand_atoms = []

for model in structure:
    for chain in model:
        for residue in chain:
            hetfield = residue.id[0]
            if hetfield == " ":  # Standard amino acid
                for atom in residue:
                    protein_atoms.append(atom)
            elif hetfield.startswith("H_") or hetfield.strip() != "":
                for atom in residue:
                    ligand_atoms.append(atom)

In [ ]:
from Bio import PDB

pdb_file = "/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/5MAJ.pdb"
output_file = "/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/protpred_only.pdb"

parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure("complex", pred_file)

io = PDB.PDBIO()

class ProteinSelect(PDB.Select):
    def accept_residue(self, residue):
        # Filter standard amino acids
        return residue.get_resname().strip() in PDB.Polypeptide.standard_aa_names

io.set_structure(structure)
io.save(output_file, ProteinSelect())

print(f"✅ Protein-only PDB written to {output_file}")


✅ Protein-only PDB written to /home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/protpred_only.pdb


In [ ]:
from Bio import PDB

pdb_file = "/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/5MAJ.pdb"
ligand_output_file = "/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/ligpred_only.pdb"

parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure("complex", pred_file)

io = PDB.PDBIO()

standard_pdb_solvents = [
    "HOH", "DOD", "H2O", "ETH", "EOH", "IPA", "MPD", "DMS",
    "ACT", "ACE", "GOL", "PEG", "SO4", "PO4", "TRS", "MES",
    "HEP", "FMT", "TLA", "EDO"
]

standard_pdb_ions = [
    "NA", "K", "CL", "CA", "MG", "ZN", "FE", "FE2", "FE3",
    "MN", "CO", "CU", "NI", "CD", "SR", "BA", "BR", "IOD",
    "CS", "RB"
]

class LigandSelect(PDB.Select):
    def accept_residue(self, residue):
        name = residue.get_resname().strip()
        standard_aa = PDB.Polypeptide.standard_aa_names
        # Exclude water and standard amino acids
        return name not in standard_aa and name not in standard_pdb_solvents and name not in standard_pdb_ions

io.set_structure(structure)
io.save(ligand_output_file, LigandSelect())

print(f"✅ Ligand-only PDB written to {ligand_output_file}")


✅ Ligand-only PDB written to /home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/ligpred_only.pdb


In [ ]:
from rdkit import Chem
from rdkit.Chem import CombineMols, rdMolTransforms

# Load separately
protein = Chem.MolFromPDBFile("/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/protpred_only.pdb", removeHs=False)
ligand = Chem.MolFromPDBFile("/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/ligpred_only.pdb", removeHs=False)

# Combine using diff function 
combined = CombineMols(protein, ligand)
conf = combined.GetConformer()

# Get number of atoms in protein
num_prot_atoms = protein.GetNumAtoms()

# Helper to find atom by name and residue number
def find_atom_index(mol, atom_name, res_num):
    for atom in mol.GetAtoms():
        info = atom.GetPDBResidueInfo()
        if info and info.GetResidueNumber() == res_num and info.GetName().strip() == atom_name:
            return atom.GetIdx()
    return None

# Get indices 
prot_idx = find_atom_index(protein, "SG", 25) 
lig_idx = find_atom_index(ligand, "C8", 1)

print(prot_idx, lig_idx)

lig_idx += num_prot_atoms # have to do this outside of fxn 

# Measure distance
dist = rdMolTransforms.GetBondLength(conf, prot_idx, lig_idx)
print(f"Measured SG–C8 distance: {dist:.2f} Å")

# can use the distance written in the pdb LINK original file (last val) to decide if the error is close to thaT or not 


191 1
Measured SG–C8 distance: 3.31 Å


In [ ]:
conf

In [ ]:
from rdkit.Chem import rdMolTransforms

# Get bonded neighbors (excluding each other)
def get_neighbor(atom, exclude_idx):
    for neighbor in atom.GetNeighbors():
        pdb_info = neighbor.GetPDBResidueInfo()
        name = pdb_info.GetName().strip()
        
        if 'C' in name:
            print(f"Neighbor name: {name}")
            if neighbor.GetIdx() != exclude_idx:
                return neighbor.GetIdx()
                
    return None

# A–B–C–D, where:
# B = SG (CYS A 25), C = C8 (7KH A 301)
atom_B = combined.GetAtomWithIdx(prot_idx)
atom_C = combined.GetAtomWithIdx(lig_idx)

# Get A and D
atom_A_idx = get_neighbor(atom_B, atom_C.GetIdx())
atom_D_idx = get_neighbor(atom_C, atom_B.GetIdx())

# Sanity check
if atom_A_idx is None or atom_D_idx is None:
    raise ValueError("Couldn't find bonded neighbor to define dihedral.")

# Measure dihedral
angle = rdMolTransforms.GetDihedralDeg(conf, atom_A_idx, prot_idx, lig_idx, atom_D_idx)
print(f"Dihedral angle (A-B-C-D): {angle:.2f}°") 


Neighbor name: CB
Neighbor name: C20
Dihedral angle (A-B-C-D): 163.66°


In [ ]:
atom_A_idx

190

In [ ]:
from pymol import cmd
cmd.load("/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_5MAJ/predictions/5MAJ/aligned_pred_orig.pdb", "mol")

angle = cmd.get_dihedral(
    "mol and chain A and resi 25 and name CB", # A 203 190 
    "mol and chain A and resi 25 and name SG", # B 204 191
    "mol and chain A and resi 1 and name C8", # C 1737 1697
    "mol and chain A and resi 1 and name C20" # D 1755 1708
)

print(angle)

CmdException:  Error: Selection 3: Not found

In [ ]:
import os
import glob
query_dir = "/home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing"
for pdb_dir in glob.glob(os.path.join(query_dir,"*/")):
        pdb_files = glob.glob(os.path.join(pdb_dir, "*.pdb"))
        pdb_file = pdb_files[0]
        print(pdb_dir)


/home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing/4EBP/
/home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing/6ALZ/
/home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing/2CDR/


In [2]:
from setup_boltz_job import create_boltz_job
pdb_file = "/home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing/6ALZ/6ALZ.pdb"
pdb_dir = "/home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing/6ALZ/"
create_boltz_job(pdb_file=pdb_file, covalent_docking=True, output_dir=pdb_dir)

Environment variable PROJECT_DIR is not set. Running setup_enviorment.sh...
Environment variables set successfully.
FASTA file created at /home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing/6ALZ/6ALZ.fasta
BKM is a ligand
YAML files created successfully: /home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing/6ALZ/6ALZ_BKM.yaml.


In [9]:
from rdkit import Chem
def find_atom_index(mol, atom_name, res_num):
    for atom in mol.GetAtoms():
        info = atom.GetPDBResidueInfo()
        if info and info.GetResidueNumber() == res_num and info.GetName().strip() == atom_name:
            return atom.GetIdx()
    return None


protein = Chem.MolFromPDBFile("/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_4EBP_CEF/predictions/4EBP_CEF/4EBP_prot.pdb")

find_atom_index(protein, 'OG', 70)

333

In [8]:
from Bio import PDB

pdb_file = "/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_4EBP_CEF/predictions/4EBP_CEF/4EBP.pdb"
output_file = "/home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_4EBP_CEF/predictions/4EBP_CEF/4EBP_prot.pdb"

parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure("complex", pdb_file)

io = PDB.PDBIO()

class ProteinSelect(PDB.Select):
    def accept_residue(self, residue):
        # Filter standard amino acids
        return residue.get_resname().strip() in PDB.Polypeptide.standard_aa_names

io.set_structure(structure)
io.save(output_file, ProteinSelect())

print(f"✅ Protein-only PDB written to {output_file}")




✅ Protein-only PDB written to /home/ymanasa/turbo/ymanasa/opt/boltz/boltz_results_4EBP_CEF/predictions/4EBP_CEF/4EBP_prot.pdb


# GETTING FASTA 

In [2]:
import requests

def fetch_fasta_from_pdb(pdb_id):
    url = f"https://www.rcsb.org/fasta/entry/{pdb_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.text
    else:
        raise ValueError(f"Failed to fetch FASTA for PDB ID {pdb_id}: {response.status_code}")

# # Example
# fasta = fetch_fasta_from_pdb("4EF9")
# print(fasta)


In [ ]:
# 1 
import csv

with open('/home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing/cov_indb2.csv', 'r') as f:
    reader = csv.DictReader(f)
    pdb_list = []
    for row in reader:
        pdb_list.append(row['PDB'])

In [49]:

len(pdb_list)

3598

In [38]:

# testing w known 
test_list = ['6ALZ','4EBP','2CDR']

In [ ]:
# 2  
fasta_list = {}

for pdb in pdb_list:
    try:
        fasta = fetch_fasta_from_pdb(pdb)
        fasta_list[pdb] = fasta
        print(pdb)
    except Exception as e:
        print(f"Failed to fetch FASTA for {pdb}: {e}")

In [ ]:
fasta_list

In [ ]:
# 3
# can use the LINK lines to find out which chain the ligand is interacting with but soemtimes the ligand is assigned to its own chain (like for peptide ligands, which we dont want)
from covalent_utils import get_link_atoms

# to get interacting chain info 
prot_chains = {}
issue_prots = []
for pdb in test_list:
    pdb_file = f"/home/ymanasa/turbo/ymanasa/opt/boltz/covalent_testing/{pdb}/{pdb}.pdb"
    try:
        prot_atom, res_name, res_num, chain_name, ccd, lig_atom, lig_idx = get_link_atoms(parent_file=pdb_file)
        prot_chains[pdb] = prot_atom, res_name, res_num, chain_name, ccd, lig_atom, lig_idx 
    except Exception as e: 
        issue_prots.append(pdb)
        print(pdb, f": {e}")



In [42]:
prot_chains

{'6ALZ': ('SG', 'CYS', 120, 'A', 'BKM', 'C01', 127),
 '4EBP': ('OG', 'SER', 41, 'B', 'CEF', 'C8', 70),
 '2CDR': ('C', 'VAL', 2, 'I', 'MY1', 'N4', 904)}

In [ ]:
# 4
# use fasta_list and prot_chains to get appropriate fasta 
errors = []
final_fasta = {}
for key, value in fasta_list.items():
    list_fasta = value.split('\n')
    new_list_fasta = []
    for item in list_fasta:
        if item.startswith('>'):
            new_list_fasta.extend([item.split('|')])  # replace with split parts
        else:
            new_list_fasta.append(item)  
    for sub_items in new_list_fasta:
        for i in sub_items:
            if 'Chain' in i:
                if prot_chains[key][3] in i: # get ligand interacting chain 
                    seq_idx = new_list_fasta.index(sub_items) + 1
    
                    if len(new_list_fasta[seq_idx]) > 20: # leaving out peptide ligands 
                        final_fasta[key] = new_list_fasta[seq_idx] 
                    else:
                        print(f'peptide ligand found or interacting chain not found for {key}')
                        errors.append(key)


peptide ligand found or interacting chain not found for 2CDR


In [46]:
final_fasta

{'6ALZ': 'GHMGSLNLDSIIGRLLEVQGSRPGKNVQLTENEIRGLCLKSREIFLSQPILLELEAPLKICGDIHGQYYDLLRLFEYGGFPPESNYLFLGDYVDRGKQSLETICLLLAYKIKYPENFFLLRGNHECASINRIYGFYDECKRRYNIKLWKTFTDCFNCLPIAAIVDEKIFCCHGGLSPDLQSMEQIRRIMRPTDVPDQGLLCDLLWSDPDKDVQGWGENDRGVSFTFGAEVVAKFLHKHDLDLICRAHQVVEDGYEFFAKRQLVTLFSAPNYCGEFDNAGAMMSVDETLMCSFQILKPAD',
 '4EBP': 'DLADRFAELERRYDARLGVYVPATGTTAAIEYRADERFAFCSTFKAPLVAAVLHQNPLTHLDKLITYTSDDIRSISPVAQQHVQTGMTIGQLCDAAIRYSDGTAANLLLADLGGPGGGTAAFTGYLRSLGDTVSRLDAEAPELNRDPPGDERDTTTPHAIALVLQQLVLGNALPPDKRALLTDWMARNTTGAKRIRAGFPADWKVIDKTGTGDYGRANDIAVVWSPTGVPYVVAVMSDRAGGGYDAEPREALLAEAATCVAGVLA'}

# FASTA SEQ in one string

In [3]:
fasta = fetch_fasta_from_pdb('2CDR')


In [12]:
list_fasta = fasta.split('\n')
final_fasta = ''
for i in list_fasta:
    if 'Chain' not in i: 
        final_fasta += i 

final_fasta


'SGISLDNSYKMDYPEMGLCIIINNKNFHKSTGMTSRSGTDVDAANLRETFRNLKYEVRNKNDLTREEIVELMRDVSKEDHSKRSSFVCVLLSHGEEGIIFGTNGPVDLKKITNFFRGDRCRSLTGKPKLFIIQACRGTELDCGIETDASGVDDDMACHKIPVEADFLYAYSTAPGYYSWRNSKDGSWFIQSLCAMLKQYADKLEFMHILTRVNRKVATEFESFSFDATFHAKKQIPCIVSMLTKELYFYHXDEVX'